# C13 project : rating prediction

Author: Ignasi

    0.1 Import package
    0.2 Describe data base 
    1.0 Feature engineering
        * Drop columns
        * transform categorical data
        * Merge data by canal
        * 
     

### 0.1 import package

In [1]:
#import package
#import pandas as pd
#import numpy as np



import warnings
warnings.filterwarnings('ignore')

# Load modules.
import os
import sys
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
from datetime import timedelta

from sklearn import linear_model, metrics, model_selection
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from statsmodels.tools.eval_measures import rmse

In [21]:
columns_name=pd.read_csv('input\columns_name.csv',sep=',').columns.tolist()

In [11]:
#df = pd.read_csv('D:\datacanal13\CSVmodelo\entradaModelo.csv', sep=',', error_bad_lines=False,header=None)
df = pd.read_csv('input\entrada_modelo.csv', sep=';', error_bad_lines=False, header=None)

In [23]:
#add the name of the columns
df.columns=columns_name

There is a column that is not clear about the information it stores, until now it will be called **"Extra"**

### 0.2 Describe data base

In [ ]:
#Describe data base
#df.describe().transpose()

In [ ]:
#len(df.CANAL.unique()), len(df.COD_UNIVERSAL.unique()),len(df.BLOQUES.unique()),len(df.PROGRAMA.unique())

Relevant information from the database:
1.  Number of tv channels: 4 
    * 1: CHV
    * 3: MEGA
    * 4: TVN
    * 5: C13
2.  Number of universal code: 1174
3.  Number of programs: 1115
4.  Number of time block: 96

The universal code is supposed to be unique, and it should have the same number of programs. In this case this does not happen.

In the database there are differences of observations per channel, this is because the start and end vary depending on the day and the channel.


In [11]:
for canal in df.CANAL.unique():
    print(df[df.CANAL==canal].shape)


(79725, 36)
(79402, 36)
(81972, 36)
(81919, 36)


## 1.0 Feature engineering

* Drop columns
    * remove columns with missing data
    * remove columns anterior y siguiente, because they are data that in practice you will never know
* Transform categorical data
    * BLOQUES
* Separate database by channel
    * Transform categorical data:  ... This is done later so as not to generate so many columns
        * PROGRAMA
    * Transform to dummy columns
        * PROGRAMA
* Merge data by canal:
    * Type of merge to be performed: Left
* Transform to dummy columns
    * BLOQUES

In [12]:
#Drop columns
df.drop(columns=["Extra",'COD_UNIVERSAL','TEMPORADA','CAPITULO','ROSTROS', 'ROSTROS_INV','INDIVIDUOHMABC25_64_ANTERIOR','INDIVIDUOHMABC25_64_SIGUIENTE' ], inplace=True)

In [13]:
# Transform categorical data
to_covert_cat = ['BLOQUES']
for col in to_covert_cat:
    df[col] = df[col].astype('category')

In [15]:
#Separate database by channel
lista_canales = df.CANAL.unique()
merge_list=['AÑO','MES','DIA','FERIADO','SEMANA_MES', 'BLOQUES']
to_covert_cat_1 = ['GENERO','SUBGENERO']
use_programa=True
list_df_canales=[]
for canal in lista_canales:
    aux=df[df.CANAL == canal].copy()
    if canal==5:
        list_programa=[]
        for lista in ['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','start_date','HORA']:
        
            list_programa.append(aux[lista].tolist())
        
    aux.drop(columns=['CANAL','PROGRAMA','start_date','HORA'], inplace=True)
    if use_programa:
        aux= pd.get_dummies(aux, columns=to_covert_cat_1)
    
    cols = aux.columns[~aux.columns.isin(merge_list)]
    aux.rename(columns = dict(zip(cols, cols + '_'+str(canal))), inplace=True)
    list_df_canales.append([aux,canal])

    
#Merge data by canal
df_final=list_df_canales[3][0]
for df_canal in list_df_canales[:-1]:
    #print(df_canal[0].shape)
    df_final=df_final.merge(df_canal[0], on=merge_list,how='left')


In [17]:
see_na=False
if see_na:
    list_columns_na=df_final.columns[df_final.isna().any()].tolist()
    print(df_final[list_columns_na].isna().sum())
    

In [18]:
#Transform to dummy columns
df_final= pd.get_dummies(df_final, columns=to_covert_cat)
df_final=df_final.sort_values(by=['AÑO','MES','SEMANA_MES','DIA'], ascending=True)
df_final=df_final.reset_index(drop=True)
df_final.fillna(0,inplace=True)

In [19]:
lista_columns = ['INDIVIDUOHMABC25_64_ACTUAL_5']

In [20]:
df_X = df_final[df_final.columns.difference(lista_columns)].copy()
df_Y = df_final[lista_columns].copy()

## Split data


In [21]:
list_tuple_week=list(set((i,j,k) for i,j,k in zip(df_final.AÑO, df_final.MES,df_final.SEMANA_MES)))
list_tuple_week.sort()

In [22]:
list_tuple_week

[(2018, 1, 1),
 (2018, 1, 2),
 (2018, 1, 3),
 (2018, 1, 4),
 (2018, 1, 5),
 (2018, 2, 1),
 (2018, 2, 2),
 (2018, 2, 3),
 (2018, 2, 4),
 (2018, 2, 5),
 (2018, 3, 1),
 (2018, 3, 2),
 (2018, 3, 3),
 (2018, 3, 4),
 (2018, 3, 5),
 (2018, 4, 1),
 (2018, 4, 2),
 (2018, 4, 3),
 (2018, 4, 4),
 (2018, 4, 5),
 (2018, 4, 6),
 (2018, 5, 1),
 (2018, 5, 2),
 (2018, 5, 3),
 (2018, 5, 4),
 (2018, 5, 5),
 (2018, 6, 1),
 (2018, 6, 2),
 (2018, 6, 3),
 (2018, 6, 4),
 (2018, 6, 5),
 (2018, 7, 1),
 (2018, 7, 2),
 (2018, 7, 3),
 (2018, 7, 4),
 (2018, 7, 5),
 (2018, 7, 6),
 (2018, 8, 1),
 (2018, 8, 2),
 (2018, 8, 3),
 (2018, 8, 4),
 (2018, 8, 5),
 (2018, 9, 1),
 (2018, 9, 2),
 (2018, 9, 3),
 (2018, 9, 4),
 (2018, 9, 5),
 (2018, 10, 1),
 (2018, 10, 2),
 (2018, 10, 3),
 (2018, 10, 4),
 (2018, 10, 5),
 (2018, 11, 1),
 (2018, 11, 2),
 (2018, 11, 3),
 (2018, 11, 4),
 (2018, 11, 5),
 (2018, 12, 1),
 (2018, 12, 2),
 (2018, 12, 3),
 (2018, 12, 4),
 (2018, 12, 5),
 (2018, 12, 6),
 (2019, 1, 1),
 (2019, 1, 2),
 (2019, 1

In [23]:
list_index=[]
for tuplas in list_tuple_week:
    list_index.append(df_final[(df_final.AÑO==tuplas[0])&(df_final.MES==tuplas[1])&(df_final.SEMANA_MES==tuplas[2])].index[-1])

In [24]:
year_train=2018
index_year_train=df_final[(df_final.AÑO==year_train)].index[-1]

In [25]:
#list_index.index(index_year_train)

62

In [19]:
# Run sliding window regression models

# Define test data
X_test = df_X.iloc[index_year_train+1:,:]
Y_test = df_Y.iloc[index_year_train+1:,:]
Yhat_test = np.zeros_like(Y_test)
Yhat_test_all = np.zeros((Yhat_test.shape[0],2))
                         
# Iterate list of index
for i in range(list_index.index(index_year_train),len(list_index)-1):
#for i in range(list_index.index(index_year_train),67):
    print("Training for test point:",list_tuple_week[i] )
    print("Test point:",list_tuple_week[i+1] )
    # Set training window
    lo = list_index[i]+1
    hi = list_index[i+1]+1
    X_train_i = df_X.iloc[:lo,:]
    Y_train_i = df_Y.iloc[:lo,:]
    X_test_i = df_X.iloc[lo:hi,:]
    
    # OLS
    reg = linear_model.LinearRegression(normalize=False).fit(X_train_i, Y_train_i)
    
    # Predict next values
    #Yhat_test_all[0,N*i:N*(i+1)] = reg.predict(X_test_i)
    Yhat_test_i = reg.predict(X_test_i)
    Yhat_test[lo-index_year_train-1:hi-index_year_train-1,:] = Yhat_test_i
    Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,0] = reg.predict(X_test_i).reshape((-1))
#     Yhat_test_all[1,N*i:N*(i+1),:] = reg.predict(X_test_i)
    print("OLS")
    # WLS
#     weights = np.tile(dem[:,0], train_win)
#     reg = linear_model.LinearRegression(normalize=True).fit(X_train_i, Y_train_i, sample_weight=weights)
    
# #     # LASSO
    #num_train = int(X_train_i.shape[0]/N)
    #lasso = linear_model.MultiTaskElasticNetCV(l1_ratio=1,normalize=True).fit(X_train_i, Y_train_i)
    #reg = MultiOutputRegressor(linear_model.Lasso(warm_start=True, alpha=lasso.alpha_, normalize=True)).fit(X_train_i, Y_train_i)
    #Yhat_test_all[1,lo-index_year_train-1:hi-index_year_train-1] = reg.predict(X_test_i).reshape((-1))
    
# #     # Ridge
#     num_train = int(X_train_i.shape[0]/N)
    #net = linear_model.MultiTaskElasticNetCV(l1_ratio=1e-8,normalize=True).fit(X_train_i, Y_train_i)
    #reg = MultiOutputRegressor(linear_model.Ridge(alpha=net.alpha_, normalize=True)).fit(X_train_i, Y_train_i)
    #Yhat_test_all[2,lo-index_year_train-1:hi-index_year_train-1]= reg.predict(X_test_i).reshape((-1))
# #     # XGBoost
    reg = MultiOutputRegressor(XGBRegressor()).fit(X_train_i, Y_train_i)
    Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,1]= reg.predict(X_test_i).reshape((-1))
    print("XGBoost")
# #     # MLP
#     reg = MLPRegressor().fit(X_train_i, Y_train_i)
#     Yhat_test_all[5,N*i:N*(i+1),:] = reg.predict(X_test_i)
    print("Index pred:{}".format((lo-index_year_train-1,hi-index_year_train-1)))
    
    

Training for test point: (2018, 12, 6)
Test point: (2019, 1, 1)
OLS
XGBoost
Index pred:(0, 465)
Training for test point: (2019, 1, 1)
Test point: (2019, 1, 2)
OLS
XGBoost
Index pred:(465, 1033)
Training for test point: (2019, 1, 2)
Test point: (2019, 1, 3)
OLS
XGBoost
Index pred:(1033, 1603)
Training for test point: (2019, 1, 3)
Test point: (2019, 1, 4)
OLS
XGBoost
Index pred:(1603, 2170)
Training for test point: (2019, 1, 4)
Test point: (2019, 1, 5)
OLS
XGBoost
Index pred:(2170, 2489)
Training for test point: (2019, 1, 5)
Test point: (2019, 2, 1)
OLS
XGBoost
Index pred:(2489, 2707)
Training for test point: (2019, 2, 1)
Test point: (2019, 2, 2)
OLS
XGBoost
Index pred:(2707, 3247)
Training for test point: (2019, 2, 2)
Test point: (2019, 2, 3)
OLS
XGBoost
Index pred:(3247, 3792)
Training for test point: (2019, 2, 3)
Test point: (2019, 2, 4)
OLS
XGBoost
Index pred:(3792, 4363)
Training for test point: (2019, 2, 4)
Test point: (2019, 2, 5)
OLS
XGBoost
Index pred:(4363, 4699)
Training for t

OLS
XGBoost
Index pred:(38159, 38694)
Training for test point: (2020, 4, 4)
Test point: (2020, 4, 5)
OLS
XGBoost
Index pred:(38694, 39003)
Training for test point: (2020, 4, 5)
Test point: (2020, 5, 1)
OLS
XGBoost
Index pred:(39003, 39226)
Training for test point: (2020, 5, 1)
Test point: (2020, 5, 2)
OLS
XGBoost
Index pred:(39226, 39762)
Training for test point: (2020, 5, 2)
Test point: (2020, 5, 3)
OLS
XGBoost
Index pred:(39762, 40298)
Training for test point: (2020, 5, 3)
Test point: (2020, 5, 4)
OLS
XGBoost
Index pred:(40298, 40832)
Training for test point: (2020, 5, 4)
Test point: (2020, 5, 5)
OLS
XGBoost
Index pred:(40832, 41373)
Training for test point: (2020, 5, 5)
Test point: (2020, 6, 1)
OLS
XGBoost
Index pred:(41373, 41899)
Training for test point: (2020, 6, 1)
Test point: (2020, 6, 2)
OLS
XGBoost
Index pred:(41899, 42432)
Training for test point: (2020, 6, 2)
Test point: (2020, 6, 3)
OLS
XGBoost
Index pred:(42432, 42971)
Training for test point: (2020, 6, 3)
Test point: (20

In [56]:
new_columns=list_programa[index_year_train+1:]


[2019, 1, 1, 1, 35, 'CUBOX']

In [87]:
Y_test.loc[:,"OLS"]=Yhat_test_all[:,0]
Y_test.loc[:,"XGB"]=Yhat_test_all[:,1]

lista_aux=['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA']

for i in range(len(lista_aux)):
    Y_test[lista_aux[i]]=list_programa[i][index_year_train+1:]

In [88]:
Y_test=Y_test[['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','INDIVIDUOHMABC25_64_ACTUAL_5','OLS',"XGB"]].copy()
Y_test['OLS'] = np.where(Y_test['OLS'] < 0, 0, Y_test['OLS'])
Y_test['XGB'] = np.where(Y_test['XGB'] < 0, 0, Y_test['XGB'])

In [89]:
Y_test#.reindex(columns=[3,0,1,2])

,AÑO,MES,SEMANA_MES,DIA,BLOQUES,PROGRAMA,INDIVIDUOHMABC25_64_ACTUAL_5,OLS,XGB
28726,2019,1,1,1,34,CUBOX,0.041340,0.000000,0.144602
28727,2019,1,1,1,35,CUBOX,0.029887,0.000000,0.127133
28728,2019,1,1,1,36,CUBOX,0.051552,0.000000,0.111829
28729,2019,1,1,1,37,CUBOX,0.010832,0.000000,0.110683
28730,2019,1,1,1,38,CUBOX,0.030159,0.000000,0.128900
...,...,...,...,...,...,...,...,...,...
81914,2020,10,5,4,105,SUPER SERIES,0.163070,0.104704,0.117785
81915,2020,10,5,4,106,SUPER SERIES,0.155364,0.137962,0.125717
81916,2020,10,5,4,107,SUPER SERIES,0.143545,0.142975,0.136993
81917,2020,10,5,4,108,SUPER SERIES,0.169082,0.131612,0.108461


In [90]:
Y_test.to_csv("model_predict_v2.csv")

In [34]:
# Run sliding window regression models

# Define test data
X_test = df_X.iloc[index_year_train+1:,:]
Y_test = df_Y.iloc[index_year_train+1:,:]
Yhat_test = np.zeros_like(Y_test)
Yhat_test_all = np.zeros((Yhat_test.shape[0],4))
                         
# Iterate list of index
for i in range(list_index.index(index_year_train),len(list_index)-1):
#for i in range(list_index.index(index_year_train),67):
    print("Training for test point:",list_tuple_week[i] )
    print("Test point:",list_tuple_week[i+1] )
    # Set training window
    lo = list_index[i]+1
    hi = list_index[i+1]+1
    X_train_i = df_X.iloc[:lo,:]
    Y_train_i = df_Y.iloc[:lo,:]
    X_test_i = df_X.iloc[lo:hi,:]
    
    
# #     # Elastic net
#   num_train = int(X_train_i.shape[0]/N)
    net = linear_model.MultiTaskElasticNetCV(l1_ratio=0.5,normalize=True).fit(X_train_i, Y_train_i)
    reg = MultiOutputRegressor(linear_model.ElasticNet(alpha=net.alpha_, normalize=True)).fit(X_train_i, Y_train_i)
    Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,2] = reg.predict(X_test_i).reshape((-1))
    print('elastic-net')
    
    #Yhat_test_all[2,lo-index_year_train-1:hi-index_year_train-1]= reg.predict(X_test_i).reshape((-1))
# #     # XGBoost
    #reg = MultiOutputRegressor(XGBRegressor()).fit(X_train_i, Y_train_i)
    #Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,1]= reg.predict(X_test_i).reshape((-1))
    #print("XGBoost")
# #     # MLP
#     reg = MLPRegressor().fit(X_train_i, Y_train_i)
#     Yhat_test_all[5,N*i:N*(i+1),:] = reg.predict(X_test_i)
    print("Index pred:{}".format((lo-index_year_train-1,hi-index_year_train-1)))
    

Training for test point: (2018, 12, 6)
Test point: (2019, 1, 1)
elastic-net
Index pred:(0, 465)
Training for test point: (2019, 1, 1)
Test point: (2019, 1, 2)
elastic-net
Index pred:(465, 1033)
Training for test point: (2019, 1, 2)
Test point: (2019, 1, 3)
elastic-net
Index pred:(1033, 1603)
Training for test point: (2019, 1, 3)
Test point: (2019, 1, 4)
elastic-net
Index pred:(1603, 2170)
Training for test point: (2019, 1, 4)
Test point: (2019, 1, 5)
elastic-net
Index pred:(2170, 2489)
Training for test point: (2019, 1, 5)
Test point: (2019, 2, 1)
elastic-net
Index pred:(2489, 2707)
Training for test point: (2019, 2, 1)
Test point: (2019, 2, 2)
elastic-net
Index pred:(2707, 3247)
Training for test point: (2019, 2, 2)
Test point: (2019, 2, 3)
elastic-net
Index pred:(3247, 3792)
Training for test point: (2019, 2, 3)
Test point: (2019, 2, 4)
elastic-net
Index pred:(3792, 4363)
Training for test point: (2019, 2, 4)
Test point: (2019, 2, 5)
elastic-net
Index pred:(4363, 4699)
Training for t

elastic-net
Index pred:(38159, 38694)
Training for test point: (2020, 4, 4)
Test point: (2020, 4, 5)
elastic-net
Index pred:(38694, 39003)
Training for test point: (2020, 4, 5)
Test point: (2020, 5, 1)
elastic-net
Index pred:(39003, 39226)
Training for test point: (2020, 5, 1)
Test point: (2020, 5, 2)
elastic-net
Index pred:(39226, 39762)
Training for test point: (2020, 5, 2)
Test point: (2020, 5, 3)
elastic-net
Index pred:(39762, 40298)
Training for test point: (2020, 5, 3)
Test point: (2020, 5, 4)
elastic-net
Index pred:(40298, 40832)
Training for test point: (2020, 5, 4)
Test point: (2020, 5, 5)
elastic-net
Index pred:(40832, 41373)
Training for test point: (2020, 5, 5)
Test point: (2020, 6, 1)
elastic-net
Index pred:(41373, 41899)
Training for test point: (2020, 6, 1)
Test point: (2020, 6, 2)
elastic-net
Index pred:(41899, 42432)
Training for test point: (2020, 6, 2)
Test point: (2020, 6, 3)
elastic-net
Index pred:(42432, 42971)
Training for test point: (2020, 6, 3)
Test point: (20

In [35]:
Y_test.loc[:,"NET"]=Yhat_test_all[:,2]

lista_aux=['AÑO','MES','SEMANA_MES','DIA','BLOQUES','PROGRAMA','start_date','HORA']

for i in range(len(lista_aux)):
    Y_test[lista_aux[i]]=list_programa[i][index_year_train+1:]

In [36]:
Y_test.head()

,INDIVIDUOHMABC25_64_ACTUAL_5,NET,AÑO,MES,SEMANA_MES,DIA,BLOQUES,PROGRAMA,start_date,HORA
28726,0.041340,0.139521,2019,1,1,1,34,CUBOX,2019-01-01,14:30
28727,0.029887,0.137616,2019,1,1,1,35,CUBOX,2019-01-01,14:45
28728,0.051552,0.131809,2019,1,1,1,36,CUBOX,2019-01-01,15:00
28729,0.010832,0.147140,2019,1,1,1,37,CUBOX,2019-01-01,15:15
28730,0.030159,0.137557,2019,1,1,1,38,CUBOX,2019-01-01,15:30


In [37]:
Y_test.to_csv("model_predict_net_v2.csv")

In [33]:
# Run sliding window regression models

# Define test data
#Yhat_test = np.zeros_like(Y_test)
Yhat_test_all_XGB = np.zeros((Yhat_test.shape[0],16))
list_errores=[]
                         
# Iterate list of index
#for i in range(list_index.index(index_year_train),len(list_index)-1):
for i in range(list_index.index(index_year_train),67):
    print("Training for test point:",list_tuple_week[i] )
    print("Test point:",list_tuple_week[i+1] )
    # Set training window
    lo = list_index[i]+1
    hi = list_index[i+1]+1
    X_train_i = df_X.iloc[:lo,:]
    Y_train_i = df_Y.iloc[:lo,:]
    X_test_i = df_X.iloc[lo:hi,:]
    Y_test_i = df_Y.iloc[lo:hi,:]
    lista_errores_semana=[]
    
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        #'min_child_weight': [1, 3, 5],
        'n_estimators' : [100, 200],
    }
    
    keys, values = zip(*param_tuning.items())
    permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
    
    for i in range(len(permutations_dicts)):
        reg = MultiOutputRegressor(XGBRegressor(**permutations_dicts[i])).fit(X_train_i, Y_train_i)
        lista_errores_semana.append(((Y_train_i - reg.predict(X_test_i).reshape((-1)) ** 2).mean())
        ista_errores_semana.append(((Y_test_i - reg.predict(X_test_i).reshape((-1)) ** 2).mean())
        Yhat_test_all[lo-index_year_train-1:hi-index_year_train-1,i]= reg.predict(X_test_i).reshape((-1))
    
    print("XGBoost")

    print("Index pred:{}".format((lo-index_year_train-1,hi-index_year_train-1)))

array([0.        , 0.        , 0.14714035, 0.        ])

# Form results dataframe
predic = np.zeros((, N, ))
actual = np.zeros((, N, ))
for t in range():
    predic[t,:,:] = Yhat_test[,:]
    actual[t,:,:] = Y_test[,:]